# Données du projet

Nous allons découvrir dans ce TP les [données utilisées](https://opendata.paris.fr/explore/dataset/horodateurs-transactions-de-paiement/) dans le projet à rendre, qui sont l'ensemble des **transactions sur les horodateurs dans la ville de Paris** sur l'année 2014. Celles-ci proviennent du site [Open Data Paris](https://opendata.paris.fr), répertoire des données ouvertes de la ville de Paris. Elles sont stockées sur le serveur *MongoDB* déjà utilisé, dans la base `horodateurs`.

Elle contient deux collections principales :

- `transactions` : ensemble des paiements
- `mobiliers`: liste de tous les horodateurs


In [1]:
import pymongo
import pandas
from pprint import pprint

con = pymongo.MongoClient("mongodb://193.51.82.104:2343/")

horo = con.horodateurs
horo.list_collection_names()

['transactions_small', 'myresults', 'transactions', 'mobiliers']

## Taille des données

Il y a un grand nombre de transactions stockées dans la base (plus de 24 millions - cf ci-dessous).

In [2]:
horo.transactions.estimated_document_count() # estimation du dénombrement (pas exacte mais bcp plus rapide)

24448030

Nous disposons d'un échantillon réduit de transactions (1%) permettant de tester rapidement le code qu'on souhaite exécuter (plutôt que de lancer un code erroné ou trop lourd sur l'ensemble des données).

In [3]:
horo.transactions_small.count_documents({}) # anciennement count()

244480

Nous disposons aussi de la liste des mobiliers, c'est-à-dire des horodateurs de la ville de Paris.

In [4]:
horo.mobiliers.count_documents({})

7810

Pour avoir une estimation de la taille des données, nous récupérons l'ensemble des transactions de l'échantillon réduit dans un `DataFrame`.

In [5]:
trans_small = pandas.DataFrame(list(horo.transactions_small.find()))

Il est maintenant possible de connaître la taille des données grâce à l'objet `values.nbytes` du `DataFrame` créé.

In [6]:
print(trans_small.values.nbytes)

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.
17602560


L'objet finalement obtenu a donc une taille de presque 17 Mo (cf calcul ci-dessous).

In [7]:
trans_small.values.nbytes / (1024 * 1024) # En Mo

16.787109375

Une rapide estimation pour la totalité (grâce à une règle de trois) nous permet de voir que l'objet final contenant les transactions sur toute l'année (sans pré-calcul dans *MongoDB*) aurait une taille de plus de 1.6 Go. Il va donc falloir faire attention aux calculs que l'on va lancer, et aux choix de récupération de données qu'on va faire.

In [8]:
trans_small.values.nbytes / (1024 ** 3) * (horo.transactions.estimated_document_count() / horo.transactions_small.estimated_document_count())

1.639368161559105

## Contenu des données

Afin de mieux comprendre les données, nous affichons le premier document de chaque collection.

Pour les transactions, nous avons les informations présentées dans le fichier PDF joint avec les données (cf lien au début du TP). On a en particulier le numéro de l'horodateur dans l'item `horodateur`. Celui-ci nous permettra de faire le lien avec l'autre collection.

In [9]:
pprint(horo.transactions.find_one())

{'_id': ObjectId('56eaa8279fae0d71f77ea2bd'),
 'date horodateur': datetime.datetime(2014, 1, 9, 14, 9, 47),
 'durée payée (h)': 0.5,
 'début stationnement': datetime.datetime(2014, 1, 9, 14, 9, 47),
 'fin stationnement': datetime.datetime(2014, 1, 9, 14, 39, 47),
 'horodateur': 1050,
 'montant carte': 1.8,
 'moyen de paiement': 'Paris Carte',
 'usager': 'Rotatif'}


Dans la collection `mobiliers`, nous avons plusieurs informations (à voir aussi dans le PD joint sur [cette page](https://opendata.paris.fr/explore/dataset/horodateurs-mobiliers/information/)). On dispose bien évidemment du numéro d'horodateurs (avec l'attribut `numhoro`), mais aussi de la localisation géographique de l'horodateur (sous différents formats, permettant leur utilisation de plusieurs façons).

In [10]:
pprint(horo.mobiliers.find_one())

{'_id': ObjectId('56eaa4552d7eb34e4432e0fb'),
 'datasetid': 'horodateurs-mobiliers',
 'fields': {'adresse': '48 vis-à-vis RUE DE LISBONNE',
            'alim': 'SOLAIRE',
            'arrondt': 8,
            'geo_point_2d': [48.877489999731374, 2.311394999713242],
            'geo_shape': {'coordinates': [2.311394999713242,
                                          48.877489999731374],
                          'type': 'Point'},
            'modele': 'SAB3',
            'numhoro': 57080603,
            'objectid': 4383,
            'regime': 'MIX',
            'tarif': '4,00 E MIX 2U',
            'tarifhor': 4.0,
            'zoneres': '8J'},
 'geometry': {'coordinates': [2.311394999713242, 48.877489999731374],
              'type': 'Point'},
 'record_timestamp': '2015-12-02T17:49:46+00:00',
 'recordid': '698c75a21ba4f4c34e72f32ea1ecd94f9de8b88c'}


## A Faire

Cette première partie à faire vous permettra de vous familariser avec les données.

### Mobiliers

1. Donner les différents types d'alimentation possible (`alim`) et le nombre d'horodateurs pour chaque. 
1. Donner le nombre de mobiliers pour chaque arrondissement (`arrondt`)
1. Lister l'ensemble des modèles existants (`modele`)
1. Croiser le régime (`regime`) et les arrondissements pour voir s'il y a des différences notables
1. Idem avec le tarif horaire (`tarifhor`)

### Transactions

1. Donner la distribution des montants payés (`montant carte`) et des durées payées (`durée payée (h)`)
1. Lister les différents moyens de paiements utilisés (`moyen de paiement`) et le type d'usager (`usager`), en ordonnant chaque table résultat par ordre décroissant du nombre de paiements
1. Déterminer s'il existe un lien entre le moyen de paiement, et le montant d'une part et la durée d'autre part

## Jointure entre collections

Il est possible de réaliser une jointure entre deux collections, dans un aggrégat, avec l'opérateur `$lookup`.

Ci-dessous, nous nous restreignons aux transactions de l'horodateur `57080603`. Ensuite, nous récupérons les informations de celui-ci dans l'autre collection (indiquée dans le `from`), via la jointure avec `$lookup`. L'attribut `localField` indique l'attribut à prendre dans la collection en cours et `foreignField` l'attribut à prendre dans la collection indiquée dans le `from`. La liste des mobiliers correspondants (1 seul normalement ici) est retournée dans un `array` nommé via le `as`. Ensuite, nous nous limitons aux 5 premiers résultats pour ne pas alourdir l'affichage.

In [11]:
res = horo.transactions_small.aggregate([
    { "$match": { "horodateur": 57080603}},
    { "$lookup": {
        "from": "mobiliers",
        "localField": "horodateur",
        "foreignField": "fields.numhoro",
        "as": "mobilier"
    }},
    { "$limit": 5}
])
liste = list(res)
pprint(liste)

[{'_id': ObjectId('56eaa9ef9fae0d71f74ec333'),
  'date horodateur': datetime.datetime(2014, 7, 10, 12, 27, 33),
  'durée payée (h)': 1.75,
  'début stationnement': datetime.datetime(2014, 7, 10, 12, 27, 33),
  'fin stationnement': datetime.datetime(2014, 7, 10, 14, 12, 33),
  'horodateur': 57080603,
  'mobilier': [{'_id': ObjectId('56eaa4552d7eb34e4432e0fb'),
                'datasetid': 'horodateurs-mobiliers',
                'fields': {'adresse': '48 vis-à-vis RUE DE LISBONNE',
                           'alim': 'SOLAIRE',
                           'arrondt': 8,
                           'geo_point_2d': [48.877489999731374,
                                            2.311394999713242],
                           'geo_shape': {'coordinates': [2.311394999713242,
                                                         48.877489999731374],
                                         'type': 'Point'},
                           'modele': 'SAB3',
                           'numhoro': 570

### A faire

1. Calculer le nombre de transactions par arrondissement
1. Calculer la répartition du type d'alimentation par arrondissement